# Regressão Linear Múltipla



## 1. Introdução

Muitos algoritmos de aprendizagem de máquinas utilização métodos de otimização. Esses algoritmos são usados por algoritmos de aprendizado de máquina para encontrar um bom conjunto de parâmetros do modelo, dado um conjunto de dados de treinamento. O algoritmo de otimização mais comum usado na aprendizagem de máquinas é o gradiente descendente estocástico. Neste tutorial, você descobrirá como implementar uma gradiente descendente estocástico para otimizar um algoritmo de regressão linear.

Ao final desta aula voce estará apto a:

1. Estimar os coeficientes de regressão linear usando a descida gradiente estocástica.
2. Fazer previsões para regressão linear multivariada.
3. Implementar regressão linear com  gradiente descendente estocástico para fazer previsões em novos dados.

### 1.1 Dataset - Seguro de Veículo Sueco

Neste tutorial, usaremos o dataset que trata da Qualidade do Vinhos (Wine Quality Data), o qual pode permitir a previsão da qualidade do vinho branco, ajudando o especialista em vinhos na avaliação de qualidade. O RMSE de linha de base do problema é de aproximadamente 0.148 pontos de qualidade. O arquivo winequality-white.csv está disponível no diretório presente. 

### 1.2 Regressão Linear Multivariada

A regressão linear é uma técnica para prever um valor real. Confusamente, esses problemas em que um valor real deve ser previsto são chamados de problemas de regressão. A regressão linear é uma técnica em que uma linha reta é usada para modelar a relação entre valores de entrada e saída. Em mais de duas dimensões, esta linha reta pode ser pensada como um plano ou hiperplano.

As previsões são feitas como uma combinação dos valores de entrada para prever o valor de saída. Cada atributo de entrada (x) é ponderado usando um coeficiente (b), e o objetivo do algoritmo de aprendizagem é descobrir um conjunto de coeficientes que resulte em boas previsões (y).


![alt text](images/regressao_linear_multipla_modelo.png "")
                             
Os coeficientes podem ser encontrados usando gradiente descendente estocástico.

### 1.3 Gradiente Descendente Estocástico

Gradiente Descendente é o processo de minimização de uma função seguindo a inclinação ou gradiente dessa função. Na aprendizagem em máquina, podemos usar uma técnica que avalie e atualize os coeficientes de cada iteração chamada gradiente descendente estocástico para minimizar o erro de um modelo em nossos dados de treinamento.

A maneira como esse algoritmo de otimização funciona é que cada instância de treinamento é submetida uma de cada vez ao modelo. O modelo faz uma previsão para uma instância de treinamento, o erro é calculado e o modelo é atualizado para reduzir o erro para a próxima previsão. Este processo é repetido para um número fixo de iterações.

Esse procedimento pode ser usado para encontrar o conjunto de coeficientes em um modelo que resulte no menor erro para o modelo nos dados de treinamento. Cada iteração, os coeficientes *(b)* são atualizados usando a equação:


![alt text](images/coeficiente_b_gradiente.png "")


Onde *b* é o coeficiente ou o peso a ser otimizado, a *taxa de aprendizado (learning rate)* é uma taxa que você deve configurar (por exemplo, 0,01), o *erro* é o erro de predição para o modelo nos dados de treinamento atribuídos ao peso e *x* é o valor de entrada.


## 2. Tutorial

Este tutorial é dividido em 3 partes:

1. Fazendo Predições.
2. Coeficientes de estimativa. 
3. Estudo de caso de qualidade do vinho.

Isso proporcionará a base que você precisa implementar e aplicar uma regressão linear com descida descendente estocástica em seus próprios problemas de modelagem preditiva.

### 2.1 Fazer previsões

O primeiro passo é desenvolver uma função que possa fazer previsões. Isso será necessário tanto na avaliação dos valores dos coeficientes do candidato quanto na aplicação do gradiente descendente estocástico. Após o modelo ser finalizado, começaremos a fazer previsões em dados de teste ou novos dados. Abaixo está uma função chamada *predict ()* que prediz um valor de saída para uma linha, dado um conjunto de coeficientes.

O primeiro coeficiente é sempre a intercepção, também chamado de viés (em ingles, bias) ou b0, pois é independente e não é responsável por um valor de entrada específico.

In [13]:
# Make a prediction with coefficients
def predict(row, coefficients):
  yhat = coefficients[0]
  for i in range(len(row)-1):
    yhat += coefficients[i + 1] * row[i]
  return yhat

### Exemplo

Dado conjunto de dados abaixo, será apresentado o uso da função predict confome exemplo que segue.

x |  y
--| -
1 | 1
2 | 3
4 | 3
3 | 2
5 | 5

Neste exemplo, existe um único valor de entrada (x) e dois valores de coeficiente (b0 e b1). A equação de predição que modelamos para este problema é:
*y = b0 + b1 × x *

Ou, com os valores de coeficientes específicos, escolhemos à mão como:
y = 0,4 + 0,8 * x 

Ao executar esta função, obtemos previsões razoavelmente próximas da saída esperada (y) valores.

In [18]:
# Example of making a prediction with coefficients
# Make a prediction
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return yhat

dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
coef = [0.4, 0.8]

for row in dataset:
  yhat = predict(row, coef)
  print("Esperado=%.3f, Predito=%.3f" % (row[-1], yhat))

Esperado=1.000, Predito=1.200
Esperado=3.000, Predito=2.000
Esperado=3.000, Predito=3.600
Esperado=2.000, Predito=2.800
Esperado=5.000, Predito=4.400


### 2.2 Estimativa de Coeficientes 

Para podermos estimar os valores dos coeficientes para nossos dados de treinamento usando gradiente descendente estocástico precisamos definir dois parâmetros:

1. Taxa de aprendizado: usado para limitar a quantidade que cada coeficiente é corrigido sempre que é atualizado.
2. Épocas: o número de vezes para percorrer os dados de treinamento ao atualizar os coeficientes.

Estes parametros, juntamente com os dados de treinamento, serão os argumentos para a função. Existem 3 loops que precisamos executar na função:

1. Loop em cada época.
2. Faça um loop sobre cada linha nos dados de treinamento para uma época.
3. Faça um loop sobre cada coeficiente e atualize-o para uma linha dos dados em uma época.

Como você pode ver, atualizamos cada coeficiente para cada linha nos dados de treinamento, em cada época. Os coeficientes são atualizados com base no erro que o modelo fez. O erro é calculado como a diferença entre a previsão feita com os coeficientes do candidato e o valor de saída esperado.

*error = prediction − expected*

Existe um coeficiente para ponderar cada atributo de entrada, e estes são atualizados de forma consistente, por exemplo:
 
b1 (t + 1) = b1 (t) - taxa de aprendizado x erro (t) x x1 (t)


O coeficiente especial no início da lista, também chamado de intercepção ou a polarização, é atualizado de forma semelhante, exceto sem uma entrada porque não está associado a um valor de entrada específico:

b0 (t + 1) = b0 (t) - taxa de aprendizagem * erro (t) 

Abaixo está uma função denominada coefficients_sgd() que calcula valores de coeficientes para um conjunto de dados de treinamento usando gradiente descendente estocástico.

Usamos uma pequena taxa de aprendizado de 0,001 e treinamos o modelo por 50 épocas, ou 50 exposições dos coeficientes para todo o conjunto de dados de treinamento. Ao executar o exemplo, imprime uma mensagem a cada época com o erro de soma quadrada para aquela época e o conjunto final de coeficientes.

In [17]:
# Estimate linear regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    print ('Coeficiente Inicial={0}' % (coef))
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            sum_error += error**2
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] - l_rate * error * row[i] 
        print(('epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error)))
    return coef

# Calculate coefficients
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
l_rate = 0.001
n_epoch = 10
coef = coefficients_sgd(dataset, l_rate, n_epoch)
print(coef)

Coeficiente Inicial={0}
epoch=0, lrate=0.001, error=46.236
epoch=1, lrate=0.001, error=41.305
epoch=2, lrate=0.001, error=36.930
epoch=3, lrate=0.001, error=33.047
epoch=4, lrate=0.001, error=29.601
epoch=5, lrate=0.001, error=26.543
epoch=6, lrate=0.001, error=23.830
epoch=7, lrate=0.001, error=21.422
epoch=8, lrate=0.001, error=19.285
epoch=9, lrate=0.001, error=17.389
[0.10710331074898374, 0.3801081881174074]
